 <h1><center>RM Triage</center></h1>

### DICOM to NIFTI conversion 
#### .json and .nii files are saved in the same folder the dcm images are located. directorio_general is a folder containing another folder with the dcm images on it. For more details clic on the link: https://drive.google.com/open?id=1aIGAJX7nv1ECq48PBeRdSWPyuclUyXl5

In [5]:
import os
from nipype.interfaces.dcm2nii import Dcm2niix

directorio_general='/home/acpuche/Documentos/scripts/Test_Triage_Project/'
folders=os.listdir(directorio_general)
for folder in folders:
    converter = Dcm2niix()
    converter.inputs.source_dir = os.path.abspath(directorio_general+'/'+folder)
    converter.inputs.output_dir = os.path.abspath(directorio_general+'/'+folder)
    converter.inputs.single_file = True
    converter.cmdline
    temp = converter.run()

200207-11:53:25,912 nipype.interface INFO:
	 stdout 2020-02-07T11:53:25.912146:Chris Rorden's dcm2niiX version v1.0.20190902  (JP2:OpenJPEG) GCC8.3.0 (64-bit Linux)
200207-11:53:25,913 nipype.interface INFO:
	 stdout 2020-02-07T11:53:25.912146:Found 26 DICOM file(s)
200207-11:53:25,914 nipype.interface INFO:
	 stdout 2020-02-07T11:53:25.912146:Philips Scaling Values RS:RI:SS = 9.08425:0:0.00576952 (see PMC3998685)
200207-11:53:25,915 nipype.interface INFO:
	 stdout 2020-02-07T11:53:25.912146:Convert 26 DICOM as /home/acpuche/Documentos/scripts/Test_Triage_Project/Flair1/Flair1_eT2_FL_AX_20151006165738_702 (288x288x26x1)
200207-11:53:25,966 nipype.interface INFO:
	 stdout 2020-02-07T11:53:25.966358:Compress: "/usr/bin/pigz" -b 960 -n -f -6 "/home/acpuche/Documentos/scripts/Test_Triage_Project/Flair1/Flair1_eT2_FL_AX_20151006165738_702.nii"
200207-11:53:25,966 nipype.interface INFO:
	 stdout 2020-02-07T11:53:25.966358:Conversion required 0.078330 seconds (0.028301 for core code).
200207-

In [13]:
%%bash
BASEDIR=/home/acpuche/Documentos/scripts/Test_Triage_Project
for folder in ${BASEDIR}/*;
    do
    mni_brain=${FSLDIR}/data/standard/MNI152_T1_2mm_brain
    nii_file=`ls /${folder}/*.nii.gz`

    in_dir=`dirname $nii_file`
    nii_file=`remove_ext $nii_file`

    #For Flair data without been previously brain extracted
    bet $nii_file ${nii_file}_brain -R -f 0.5 -g 0 -m
    nii_file=${nii_file}_brain
    mask_file=${nii_file}_mask
    fslmaths $mask_file -eroF $mask_file
    csf_thr=`fslstats $nii_file -k $mask_file -s`
    fslmaths $nii_file -mas $mask_file -uthr $csf_thr -dilM -bin ${nii_file}_csf_mask
    fslmaths $mask_file -mas $mask_file -sub ${nii_file}_csf_mask -bin $mask_file
    wait

    #spatial smoothing using a Gauss filter FWHM=2.355 sigma
    fslmaths $nii_file -s 1.3 $nii_file 
    #Registration to standard space
    flirt -in $nii_file -ref $mni_brain -out ${nii_file}_mni-2mm -omat ${nii_file}_mni-2mm.mat -bins 256 -cost corratio -searchrx -90 90 -searchry -90 90 -searchrz -90 90 -dof 12  -interp trilinear
    nii_file2=${nii_file}_mni-2mm
    # Apply transformation to the mask
    flirt -in $mask_file -ref $mni_brain -out $mask_file -init ${nii_file}_mni-2mm.mat -applyxfm


    # Splitting of both hemispheres 
    fslroi ${nii_file2} ${nii_file}_r 0 45 0 -1 0 -1 
    fslroi ${nii_file2} ${nii_file}_l 45 45 0 -1 0 -1
    fslroi $mask_file ${mask_file}_r 0 45 0 -1 0 -1
    fslroi $mask_file ${mask_file}_l 45 45 0 -1 0 -1
    fslswapdim ${nii_file}_l -x y z ${nii_file}_l_flipped
    fslswapdim ${mask_file}_l -x y z ${mask_file}_l_flipped
    fslmaths ${mask_file}_r -mul ${mask_file}_l_flipped -bin ${mask_file}_no_csf
done


          Will use voxel-based orientation which is probably incorrect - *PLEASE CHECK*!


          Will use voxel-based orientation which is probably incorrect - *PLEASE CHECK*!

